In [2]:
import pytz
from datetime import datetime
import pandas as pd
import numpy as np
import os
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import keras
import pandas as pd
from itables import show
from keras.models import Sequential # type: ignore
from keras.layers import Dense, Input, LSTM, Embedding, SimpleRNN, GRU, Bidirectional, Conv1D, MaxPooling1D, Flatten, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.models import load_model
from keras.metrics import MeanSquaredError, MeanAbsoluteError, RootMeanSquaredError       
from keras.regularizers import l2
from sklearn.metrics import mean_squared_error, mean_absolute_error, root_mean_squared_error, mean_absolute_percentage_error
import scipy
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sktime.transformations.series.holiday import HolidayFeatures
from holidays import country_holidays, financial_holidays
from sklearn.multioutput import MultiOutputRegressor
from lightgbm import LGBMRegressor
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from padasip.filters import FilterRLS
import pickle
from sklearn.model_selection import TimeSeriesSplit
np.printoptions(suppress=True)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
from weather_utils import WeatherUtils
import seaborn as sns
import holidays
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 1000)


In [3]:
saved_ann_models_path = './../Saved_Models/ANN_Saved_Models'
saved_lstm_models_path = './../Saved_Models/LSTM_Saved_Models'
saved_rls_combiner_models_path = './../Saved_Models/RLS_Combiner_Saved_Models'
feeders_metadata_path = './../Data/Filtered_Feeders_Metadata/Final_Selected_Feeders_Data_with_Coordinates.csv'
feeders_root_path = "./../Data/Feeder_Data/"
openmeteo_weather_data_path = "./../Data/Feeder_Weather_Combined_Data/"
filtered_feeders_metadata_file_path = "../Data/Filtered_Feeders_Metadata/Final_Selected_Feeders_Data_with_Coordinates.csv"
weather_data_path = "./../Data/Weather_Data/high_resolution_weather_arch_hall.csv"
saved_models_path = "./../Saved_Models"
saved_results_path = "./../Data/Results"
train_stats_path = './../Data/Filtered_Feeders_Metadata/Train_Stats'

end_train = "2024-06-16 23:59:59"
end_val = "2024-06-30 23:59:59"
end_test = "2024-07-31 23:59:59"

start_train_date = "2024-01-01"
start_val_date = "2024-06-17"
start_test_date = "2024-07-01"
end_train_date = "2024-06-16"
end_val_date = "2024-06-30"
end_test_date = "2024-07-31"

freq = "h"

In [4]:
threshold = 6

def remove_first_and_last_days(raw_data):
    raw_data = raw_data.copy()
    first_day = raw_data.index[0].strftime("%Y-%m-%d")
    last_day = raw_data.index[-1].strftime("%Y-%m-%d")
    
    raw_data = raw_data.drop(raw_data.loc[first_day].index)
    raw_data = raw_data.drop(raw_data.loc[last_day].index)
    
    return raw_data

def remove_last_day(raw_data):
    raw_data = raw_data.copy()
    # first_day = raw_data.index[0].strftime("%Y-%m-%d")
    last_day = raw_data.index[-1].strftime("%Y-%m-%d")
    
    # raw_data = raw_data.drop(raw_data.loc[first_day].index)
    raw_data = raw_data.drop(raw_data.loc[last_day].index)
    
    return raw_data

def add_extra_hour_in_beginning(raw_data):
    raw_data = raw_data.copy()
    first_index = raw_data.index[0]
    hour_before_first_index = first_index - pd.Timedelta(hours=1)
    hour_before_first_index = pd.Timestamp(hour_before_first_index)
    raw_data.loc[hour_before_first_index] = raw_data.loc[first_index]
    raw_data = raw_data.sort_index()
    
    return raw_data

def remove_incomplete_days(raw_data):
    raw_data = raw_data.copy()
    grouped_raw_data = raw_data.groupby(by=raw_data.index.date).count()
    incomplete_days = grouped_raw_data[grouped_raw_data != 24].index
    raw_data = remove_dates_from_raw_data(raw_data, incomplete_days, "Removing Incomplete Days")
    
    return raw_data

# This function assumes that the incoming raw data has a datetime index
def preprocess_raw_data(data, column_name):
    data = data.copy()
    data = add_extra_hour_in_beginning(data)
    data = data.resample(rule='h', closed='left', label='right').mean()
    data = remove_last_day(data)
    data = data[column_name]
    filtered_data, nan_values_filtered = remove_nan_days_above_threshold(threshold, data)
    filtered_data = filtered_data.interpolate(method='time')
    filtered_data = remove_incomplete_days(filtered_data)
    
    
    return filtered_data

def remove_dates_from_raw_data(raw_data, dates, comment):
    print(f"{comment} - Total Dropped Days: ", len(dates))    
    filtered_data = raw_data.copy()

    for date in dates:
        date = date.strftime("%Y-%m-%d")
        filtered_data = filtered_data.drop(filtered_data.loc[date].index)

    return filtered_data

def remove_nan_days_above_threshold(threshold, raw_data):
    nan_values = raw_data[raw_data.isna()].fillna(0)
    nan_values = nan_values.groupby(by=nan_values.index.date).count()
    nan_values.index = pd.to_datetime(nan_values.index)
    nan_values_filtered = nan_values[nan_values > threshold]
    filtered_data = remove_dates_from_raw_data(raw_data, nan_values_filtered.index, "Removing Nan Values")
    

    return filtered_data, nan_values_filtered

def remove_space_in_column_names(data):
    data.columns = data.columns.str.replace(' ', '_')
    return data

def get_raw_data_from_path(filepath):
    raw_data = pd.read_csv(
        filepath,
        na_values=[""],
        skipinitialspace=True,
        index_col='Time',
        parse_dates=True,
    ).dropna()
    
    raw_data.index = pd.to_datetime(raw_data.index).tz_localize(None)
    raw_data = raw_data[~raw_data.index.duplicated(keep='last')]

    return raw_data

def minmax_scale(data, min, max):
    return (data - min) / (max - min)

def minmax_inverse_scale(data, min, max):
    return (data * (max - min)) + min


In [5]:
holiday_years=[2020, 2021, 2022, 2023, 2024, 2025, 2026, 2027, 2028, 2029, 2030]

def get_barbados_holidays_list(years):
    barbados_holidays_list = holidays.Barbados(years=years)
    barbados_holidays_df = pd.DataFrame(barbados_holidays_list.values(), index=pd.to_datetime(list(barbados_holidays_list.keys())))
    return barbados_holidays_df

def get_unique_dates_from_hourly_data(data):
    return pd.to_datetime(data.index.date).unique()

def get_one_hot_day_of_week(dates_index):
    day_of_week = pd.get_dummies(dates_index.dayofweek).astype(int)
    day_of_week.index = dates_index
    day_of_week.columns = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    return day_of_week

def add_is_holiday_column_to_one_hot_day_of_week(day_of_week, barbados_holidays_df):
    day_of_week['is_holiday'] = day_of_week.index.isin(barbados_holidays_df.index).astype(int)
    return day_of_week

barbados_holidays_df = get_barbados_holidays_list(holiday_years)
barbados_holidays_df


,0
2020-01-01,New Year's Day
2020-01-21,Errol Barrow Day
2020-04-10,Good Friday
2020-04-13,Easter Monday
2020-04-28,National Heroes Day
...,...
2030-08-01,Emancipation Day
2030-08-05,Kadooment Day
2030-11-30,Independence Day
2030-12-25,Christmas Day


In [6]:
hours_in_day = 15
target_column = 'NetLoadDemand'

def reshape_X_to_LSTM_shape(X):
    return X.reshape(X.shape[0], 1, X.shape[1])

def create_X_y(data, target_column, hours_in_day, day_of_week):
    X = []
    y_bl = []
    y_cl = []
    
    prev_dates = []
    cur_dates = []
    
    for i in range(hours_in_day, len(data), hours_in_day):
        cur = i
        prev = i - hours_in_day
        
        if(data.index[cur] != data.index[prev] + pd.Timedelta(days=1)):
            print(f'ALERT: Current: {data.index[cur]} to {data.index[cur + hours_in_day - 1]}, Previous: {data.index[prev]} to {data.index[prev + hours_in_day - 1]}')
            print('ALERT: Day Change Detected')
            continue
        
        print(f'Current: {data.index[cur]} to {data.index[cur + hours_in_day - 1]}, Previous: {data.index[prev]} to {data.index[prev + hours_in_day - 1]}')
        
        prev_historic_temp = data['temperature_2m_historic'].iloc[prev:cur].values.flatten()
        prev_forecasted_temp = data['temperature_2m_forecast'][prev:cur].values.flatten()
        prev_historic_shortwave_radiation = data['shortwave_radiation_historic'].iloc[prev:cur].values.flatten()
        prev_forecasted_shortwave_radiation = data['shortwave_radiation_forecast'][prev:cur].values.flatten()
        prev_load_demand = data[target_column].iloc[prev:cur].values.flatten()
        
        cur_historic_temp = data['temperature_2m_historic'].iloc[cur:cur + hours_in_day].values.flatten()
        cur_forecasted_temp = data['temperature_2m_forecast'].iloc[cur:cur + hours_in_day].values.flatten()
        cur_historic_shortwave_radiation = data['shortwave_radiation_historic'].iloc[cur:cur + hours_in_day].values.flatten()
        cur_forecasted_shortwave_radiation = data['shortwave_radiation_forecast'].iloc[cur:cur + hours_in_day].values.flatten()
        cur_load_demand = data[target_column].iloc[cur:cur + hours_in_day].values.flatten()
        cur_day_of_week = day_of_week.iloc[cur//hours_in_day].values.flatten()
        
        X_day = np.hstack([prev_load_demand, prev_historic_temp, prev_forecasted_temp, cur_historic_temp, cur_forecasted_temp, prev_historic_shortwave_radiation, prev_forecasted_shortwave_radiation, cur_historic_shortwave_radiation, cur_forecasted_shortwave_radiation, cur_day_of_week])
        y_day_bl = cur_load_demand
        y_day_cl = cur_load_demand - prev_load_demand
        
        X.append(X_day)
        y_bl.append(y_day_bl)
        y_cl.append(y_day_cl)
        
        prev_day_date = data.index[prev:cur].values.flatten()
        cur_day_date = data.index[cur:cur + hours_in_day].values.flatten()
        
        prev_dates.append(prev_day_date)
        cur_dates.append(cur_day_date)
    
    
    X = np.array(X)
    X_LSTM = reshape_X_to_LSTM_shape(X)
    y_bl = np.array(y_bl)
    y_cl = np.array(y_cl)
    
    cur_dates = np.array(cur_dates)
    prev_dates = np.array(prev_dates)
    
    return X, X_LSTM, y_bl, y_cl, cur_dates, prev_dates

def convert_cl_to_bl(y_pred_cl, y, X):
    y_prev_bl = X[:, :hours_in_day]
    # y_pred_cl_bl = []
    
    print(y_prev_bl.shape, y_pred_cl.shape)
    
    if(y_prev_bl.shape == y_pred_cl.shape):
        y_pred_cl_bl = y_pred_cl + y_prev_bl
    
    return y_pred_cl_bl


In [7]:
# callbacks=[EarlyStopping(patience=50), ReduceLROnPlateau(patience=20, factor=0.1, min_lr=0.000000001)]

def predict_model(model, X):
    y_pred = model.predict(X)
    return y_pred

def get_trained_model(path, model_type, feeder_filesave_name):
    model_save_path = f'{path}/{model_type}/{feeder_filesave_name}.keras'
    model = load_model(model_save_path)
    
    return model

def get_train_stats(train_stats_path, feeder_filesave_name):
    train_stats_file_path = f'{train_stats_path}/{feeder_filesave_name}_train_stats.csv'
    train_stats = pd.read_csv(train_stats_file_path, index_col=0)
    
    return train_stats

def adapt_rls_combiner_and_predict(results, hours, rls_combiner=None, adapt=True):
    rls_preds = []
    samples = results.shape[0]
    # hours = y.shape[1]

    if not rls_combiner:
        print("Building a new RLS Combiner")
        rls_combiner = [FilterRLS(n=results.shape[1] - 1, mu=0.99) for _ in range(hours)]

    for i in range(0, samples, hours):
        for j in range(hours):
            rls_train_sample = [results.iloc[i][col] for col in results.columns[1:]]
            rls_train_sample = np.array(rls_train_sample).reshape(-1)
            rls_actual_sample = results.iloc[i, 0]

            rls_preds_sample = rls_combiner[j].predict(rls_train_sample)
            if(adapt):
              rls_combiner[j].adapt(rls_actual_sample, rls_train_sample)
            rls_preds.append(rls_preds_sample)

            i += 1

    results["RLS"] = rls_preds
    return results, rls_combiner

def build_rls_combiner(hours_in_day, n_preds):
    return [FilterRLS(n=n_preds, mu=0.99) for _ in range(hours_in_day)]

def adapt_and_predict_rls_combiner(actual, pred_1, pred_2, hours_in_day, rls_combiner=None):
    if not rls_combiner:
        rls_combiner = build_rls_combiner(hours_in_day, 2)
    
    samples = actual.shape[0]
    rls_preds = []
    
    for i in range(samples):
        for j in range(hours_in_day):
            rls_train_sample = [pred_1[i, j], pred_2[i, j]]
            rls_train_sample = np.array(rls_train_sample).reshape(-1)
            rls_actual_sample = actual[i, j]

            rls_preds_sample = rls_combiner[j].predict(rls_train_sample)
            rls_combiner[j].adapt(rls_actual_sample, rls_train_sample)
            rls_preds.append(rls_preds_sample)

            # i += 1
            
    rls_preds = np.array(rls_preds).reshape(samples, hours_in_day)
            
    return rls_preds, rls_combiner

In [8]:
feeders_metadata = pd.read_csv(feeders_metadata_path)
weather_utils = WeatherUtils()

for i in range(feeders_metadata.shape[0] - 8):
    feeder_metadata = feeders_metadata.iloc[i]
    feeder_name = feeder_metadata['FeederName']
    feeder_data_filename = feeder_metadata['FileName']
    feeder_filesave_name = feeder_metadata['FileSaveName']
    feeder_data_path = feeders_root_path + feeder_data_filename
    ann_save_path = f"{saved_models_path}/ANN_Saved_Models/{feeder_filesave_name}_ANN_Model.keras"
    lstm_save_path = f"{saved_models_path}/LSTM_Saved_Models/{feeder_filesave_name}_LSTM_Model.keras"
    rls_combiner_save_path = f"{saved_models_path}/RLS_Combiner_Saved_Models/{feeder_filesave_name}_RLS_Combiner.pkl"
    feeder_data = get_raw_data_from_path(feeder_data_path)
    net_load_demand = preprocess_raw_data(feeder_data, "NetLoadDemand")
    latitude = feeder_metadata['Latitude']
    longitude = feeder_metadata['Longitude']
    
    
    print("========================================")
    print(feeder_name)
    print("========================================")
    
    historic_weather_data = weather_utils.fetch_historic_data_from_api(latitude, longitude, start_val_date, end_val_date)
    forecast_weather_data = weather_utils.fetch_forecast_data_from_api(latitude, longitude, start_val_date, end_val_date)
    
    temperature_historic = historic_weather_data["temperature_2m_historic"]
    temperature_forecast = forecast_weather_data["temperature_2m_forecast"]
    shortwave_radiation_historic = historic_weather_data["shortwave_radiation_historic"]
    shortwave_radiation_forecast = forecast_weather_data["shortwave_radiation_forecast"]
    
    
    combined_data = pd.concat([net_load_demand, temperature_historic, temperature_forecast, shortwave_radiation_historic, shortwave_radiation_forecast], axis=1).dropna()
    combined_data = combined_data.loc[start_val_date:end_val]
    combined_data_daytime = combined_data.between_time('06:00', '20:00')
    
    train_stats = get_train_stats(train_stats_path, feeder_filesave_name)
    print(train_stats)
    
    normalized_data = minmax_scale(combined_data_daytime, train_stats['Min'], train_stats['Max'])
    print(normalized_data.describe())
    
    denormalized_data = minmax_inverse_scale(normalized_data, train_stats['Min'], train_stats['Max'])
    # print(denormalized_data.describe())
    
    dates_index = get_unique_dates_from_hourly_data(normalized_data)
    day_of_week = get_one_hot_day_of_week(dates_index)
    day_of_week = add_is_holiday_column_to_one_hot_day_of_week(day_of_week, barbados_holidays_df)
    
    X_val, X_val_LSTM, y_val_bl, y_val_cl, cur_dates_val, prev_dates_val = create_X_y(normalized_data, target_column, hours_in_day, day_of_week)
    
    trained_ann_model_bl = get_trained_model(saved_models_path, "ANN_BL", feeder_filesave_name)
    trained_ann_model_cl = get_trained_model(saved_models_path, "ANN_CL", feeder_filesave_name)
    trained_lstm_model_bl = get_trained_model(saved_models_path, "LSTM_BL", feeder_filesave_name)
    trained_lstm_model_cl = get_trained_model(saved_models_path, "LSTM_CL", feeder_filesave_name)
    
    
    y_pred_val_bl = predict_model(trained_ann_model_bl, X_val)
    y_pred_val_cl = predict_model(trained_ann_model_cl, X_val)
    y_pred_val_bl_LSTM = predict_model(trained_lstm_model_bl, X_val_LSTM)
    y_pred_val_cl_LSTM = predict_model(trained_lstm_model_cl, X_val_LSTM)
    
    y_pred_val_bl_cl = convert_cl_to_bl(y_pred_val_cl, y_val_bl, X_val)
    y_pred_val_bl_cl_LSTM = convert_cl_to_bl(y_pred_val_cl_LSTM, y_val_bl, X_val)
    
    print(X_val.shape, X_val_LSTM.shape, y_val_bl.shape, y_val_cl.shape, cur_dates_val.shape, prev_dates_val.shape)
    print(y_pred_val_bl.shape, y_pred_val_cl.shape, y_pred_val_bl_LSTM.shape, y_pred_val_cl_LSTM.shape)
    
    y_pred_ann_rls, rls_combiner_ann = adapt_and_predict_rls_combiner(y_val_bl, y_pred_val_bl, y_pred_val_bl_cl, hours_in_day)
    y_pred_lstm_rls, rls_combiner_lstm = adapt_and_predict_rls_combiner(y_val_bl, y_pred_val_bl_LSTM, y_pred_val_bl_cl_LSTM, hours_in_day)
    y_pred_final_rls, rls_combiner_final = adapt_and_predict_rls_combiner(y_val_bl, y_pred_ann_rls, y_pred_lstm_rls, hours_in_day)
    
    final_results = pd.DataFrame([y_val_bl.flatten(), y_pred_ann_rls.flatten(), y_pred_lstm_rls.flatten(), y_pred_final_rls.flatten()]).T
    final_results.columns = ["Actual", "ANN_RLS", "LSTM_RLS", "Final_RLS"]
    final_results.index = cur_dates_val.flatten()
    final_results = final_results.apply(lambda x: minmax_inverse_scale(x, train_stats.loc[target_column, 'Min'], train_stats.loc[target_column, 'Max']))
    
    px.line(final_results.loc['2024-06-21':]).show()


# arch_hall_feeder_data_path = feeders_root_path + "Arch_Hall_ST2B13.csv"
# arch_hall_feeder_data = get_raw_data_from_path(arch_hall_feeder_data_path)
# arch_hall_feeder_data = add_extra_hour_in_beginning(arch_hall_feeder_data)
# arch_hall_net_load_demand = preprocess_raw_data(arch_hall_feeder_data, "NetLoadDemand")
# arch_hall_net_load_demand

Removing Nan Values - Total Dropped Days:  3
Removing Incomplete Days - Total Dropped Days:  0
Arch Hall
Coordinates 13.181018829345703°N -59.56243896484375°E
Elevation 114.0 m asl
Timezone b'America/Barbados' b'GMT-4'
Timezone difference to GMT+0 -14400 s
Coordinates 13.125°N -59.625°E
Elevation 114.0 m asl
Timezone b'America/Barbados' b'GMT-4'
Timezone difference to GMT+0 -14400 s
                                Mean     Std      Min      Max
NetLoadDemand                3037.37 3151.18 -4721.06 10984.23
temperature_2m_historic        27.78    1.94    22.30    32.40
temperature_2m_forecast        27.90    1.56    24.08    31.73
shortwave_radiation_historic  400.14  322.29     0.00  1026.00
shortwave_radiation_forecast  404.40  321.86     0.00   987.00
       NetLoadDemand  temperature_2m_historic  temperature_2m_forecast  shortwave_radiation_historic  shortwave_radiation_forecast
count         210.00                   210.00                   210.00                        210.00     

Removing Nan Values - Total Dropped Days:  2
Removing Incomplete Days - Total Dropped Days:  0
Chapel Street
Coordinates 13.11072063446045°N -59.590179443359375°E
Elevation 6.0 m asl
Timezone b'America/Barbados' b'GMT-4'
Timezone difference to GMT+0 -14400 s
Coordinates 13.125°N -59.625°E
Elevation 6.0 m asl
Timezone b'America/Barbados' b'GMT-4'
Timezone difference to GMT+0 -14400 s
                               Mean    Std    Min     Max
NetLoadDemand                914.80 419.45 264.00 1572.00
temperature_2m_historic       28.71   1.93  23.30   33.20
temperature_2m_forecast       28.61   1.56  24.78   32.43
shortwave_radiation_historic 402.00 323.76   0.00 1022.00
shortwave_radiation_forecast 404.57 322.15   0.00  987.00
       NetLoadDemand  temperature_2m_historic  temperature_2m_forecast  shortwave_radiation_historic  shortwave_radiation_forecast
count         210.00                   210.00                   210.00                        210.00                        210.00
mean

Removing Nan Values - Total Dropped Days:  2
Removing Incomplete Days - Total Dropped Days:  0
Dukes
Coordinates 13.181018829345703°N -59.56243896484375°E
Elevation 249.0 m asl
Timezone b'America/Barbados' b'GMT-4'
Timezone difference to GMT+0 -14400 s
Coordinates 13.25°N -59.625°E
Elevation 249.0 m asl
Timezone b'America/Barbados' b'GMT-4'
Timezone difference to GMT+0 -14400 s
                                Mean    Std     Min     Max
NetLoadDemand                1241.52 646.49 -300.83 4017.22
temperature_2m_historic        26.91   1.94   21.42   31.52
temperature_2m_forecast        26.94   1.74   22.31   31.46
shortwave_radiation_historic  399.68 321.85    0.00 1026.00
shortwave_radiation_forecast  398.44 318.55    0.00  986.00
       NetLoadDemand  temperature_2m_historic  temperature_2m_forecast  shortwave_radiation_historic  shortwave_radiation_forecast
count         210.00                   210.00                   210.00                        210.00                        210.

Removing Nan Values - Total Dropped Days:  3
Removing Incomplete Days - Total Dropped Days:  0
Fontabelle 1
Coordinates 13.11072063446045°N -59.590179443359375°E
Elevation 6.0 m asl
Timezone b'America/Barbados' b'GMT-4'
Timezone difference to GMT+0 -14400 s
Coordinates 13.125°N -59.625°E
Elevation 6.0 m asl
Timezone b'America/Barbados' b'GMT-4'
Timezone difference to GMT+0 -14400 s
                               Mean    Std   Min     Max
NetLoadDemand                 14.45   5.46 -2.60   27.67
temperature_2m_historic       28.71   1.93 23.30   33.20
temperature_2m_forecast       28.61   1.56 24.78   32.43
shortwave_radiation_historic 402.00 323.76  0.00 1022.00
shortwave_radiation_forecast 404.57 322.15  0.00  987.00
       NetLoadDemand  temperature_2m_historic  temperature_2m_forecast  shortwave_radiation_historic  shortwave_radiation_forecast
count         210.00                   210.00                   210.00                        210.00                        210.00
mean       

In [9]:
pd.DataFrame([y_val_bl.flatten(), y_pred_val_bl.flatten(), y_pred_val_bl_LSTM.flatten()]).T

,0,1,2
0,0.61,0.55,0.60
1,0.60,0.74,0.61
2,0.66,0.76,0.64
3,0.75,0.70,0.63
4,0.75,0.66,0.62
...,...,...,...
190,0.27,0.43,0.41
191,0.33,0.57,0.49
192,0.43,0.60,0.55
193,0.42,0.61,0.60
